Neural network 
=============

This file contains lot's (and not even all) of the variations tried for the neural network.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import sys 
import os
sys.path.append(os.path.abspath("/media/sf_Project2/Code"))
from IO import Input
from IO import Output
import pandas as pd

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [2]:
with cd("/media/sf_Project2/Code"): 
    train_dataset = np.array(Input.load_trainset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    train_labels = np.array(Input.load_trainset_labels()).astype('float32')
    valid_dataset = np.array(Input.load_validationset_caffefeatures(featureSelectionMethod='RF',Percentile = 100)).astype('float32')
    valid_labels = np.array(Input.load_validationset_labels()).astype('float32')

In [12]:
with cd("/media/sf_Project2/Code"): 
    #To be safe, do this in 3 parts from now on!!!
    #test_data = np.array(Input.load_testdata_caffefeatures(True,range(38000),'RF',100)).astype('float32')
    test_data = np.array(Input.load_testdata_caffefeatures(True,range(38000,80000),'RF',100)).astype('float32')

In [13]:
print(test_data.shape)

(41726, 3983)


In [3]:
num_labels=10
train_labels = np.squeeze((np.arange(num_labels) == train_labels[:,None]).astype(np.float32))
valid_labels = np.squeeze((np.arange(num_labels) == valid_labels[:,None]).astype(np.float32))
train_labels0 = train_labels[:,1]
train_labels0 = train_labels0.reshape((train_labels.shape[0],1))

print(train_labels0)
#print(train_dataset)
#train_labels = train_labels.reshape((train_labels.shape[0],1))
#valid_labels = valid_labels.reshape((valid_labels.shape[0],1))
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
feature_size = train_dataset.shape[1]
print(train_labels)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Training set (14363, 3983) (14363, 10)
Validation set (8061, 3983) (8061, 10)
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [60]:
#image_size = 28
#num_labels = 10
#num_channels = 1 # grayscale

#import numpy as np

#def reformat(dataset, labels):
#  dataset = dataset.reshape(
#    (-1, image_size, image_size, num_channels)).astype(np.float32)
#  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
#  return dataset, labels
#train_dataset, train_labels = reformat(train_dataset, train_labels)
#valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
#test_dataset, test_labels = reformat(test_dataset, test_labels)
#print('Training set', train_dataset.shape, train_labels.shape)
#print('Validation set', valid_dataset.shape, valid_labels.shape)
#print('Test set', test_dataset.shape, test_labels.shape)

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels,1))
          / predictions.shape[0])

In [64]:
def singleAccuracy(predictions, labels):
  return (100.0 * np.sum(predictions == labels)
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [21]:
batch_size = 16
hlSize1 = 2048
hlSize2 = 1024
hlSize3 = 512
hlSize4 = num_labels
beta = 0.0001
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.int64, shape=(batch_size))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize1],
            stddev=0.01))
  input_biases = tf.Variable(tf.zeros([hlSize1]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize1,hlSize2], 
            stddev=0.01))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[hlSize2]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [hlSize2,hlSize3], 
            stddev=0.01))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[hlSize3]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [hlSize3,hlSize4], 
            stddev=0.01))
  layer3_biases = tf.Variable(tf.constant(0.0, shape=[hlSize4]))
  output_weights = tf.Variable(tf.truncated_normal(
      [hlSize4,num_labels], 
            stddev=0.01))
  output_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
    
  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.nn.relu(tf.matmul(layer1, layer1_weights) + layer1_biases)
    layer3 = tf.nn.relu(tf.matmul(layer2, layer2_weights) + layer2_biases)
    layer4 = tf.nn.relu(tf.matmul(layer3, layer3_weights) + layer3_biases)
    layer5 = tf.matmul(layer4, output_weights) + output_biases
    return layer4
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #loss = loss + beta * tf.nn.l2_loss(input_weights) + \
  #  beta * tf.nn.l2_loss(layer1_weights) + \
  #  beta * tf.nn.l2_loss(layer2_weights) + \
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [22]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")

Initialized
Minibatch loss at step 0: 2.304760
Minibatch accuracy: 0.0%
Validation accuracy: 10.8%
Minibatch loss at step 100: 2.031671
Minibatch accuracy: 100.0%
Validation accuracy: 10.7%
Minibatch loss at step 200: 2.302646
Minibatch accuracy: 0.0%
Validation accuracy: 10.8%
Minibatch loss at step 300: 2.004498
Minibatch accuracy: 100.0%
Validation accuracy: 10.7%
Minibatch loss at step 400: 2.327179
Minibatch accuracy: 0.0%
Validation accuracy: 10.3%
Minibatch loss at step 500: 2.302585
Minibatch accuracy: 0.0%
Validation accuracy: 10.8%
Minibatch loss at step 600: 2.326915
Minibatch accuracy: 0.0%
Validation accuracy: 10.5%
Minibatch loss at step 700: 2.329764
Minibatch accuracy: 0.0%
Validation accuracy: 11.8%
Minibatch loss at step 800: 2.354721
Minibatch accuracy: 0.0%
Validation accuracy: 10.5%
Minibatch loss at step 900: 1.586996
Minibatch accuracy: 100.0%
Validation accuracy: 10.7%
Minibatch loss at step 1000: 2.372885
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
fini

In [115]:
batch_size = 16
hlSize1 = 256
beta = 0.0001
global_step = tf.Variable(0)  # count the number of steps taken.
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize1],
            stddev=0.01))
  input_biases = tf.Variable(tf.zeros([hlSize1]))
  output_weights = tf.Variable(tf.truncated_normal(
      [hlSize1,num_labels], 
            stddev=0.01))
  output_biases = tf.Variable(tf.constant(0.5, shape=[num_labels]))
    
  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, output_weights) + output_biases
    return layer2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [116]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")

Initialized
Minibatch loss at step 0: 2.278514
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 100: 0.651811
Minibatch accuracy: 100.0%
Validation accuracy: 10.7%
Minibatch loss at step 200: 2.038844
Minibatch accuracy: 100.0%
Validation accuracy: 8.9%
Minibatch loss at step 300: 0.407706
Minibatch accuracy: 100.0%
Validation accuracy: 10.7%
Minibatch loss at step 400: 4.995739
Minibatch accuracy: 12.5%
Validation accuracy: 10.8%
Minibatch loss at step 500: 2.960555
Minibatch accuracy: 37.5%
Validation accuracy: 8.9%
Minibatch loss at step 600: 1.966929
Minibatch accuracy: 0.0%
Validation accuracy: 12.0%
Minibatch loss at step 700: 3.672642
Minibatch accuracy: 0.0%
Validation accuracy: 8.6%
Minibatch loss at step 800: 0.090602
Minibatch accuracy: 100.0%
Validation accuracy: 8.9%
Minibatch loss at step 900: 0.293407
Minibatch accuracy: 100.0%
Validation accuracy: 10.7%
Minibatch loss at step 1000: 3.027869
Minibatch accuracy: 0.0%
Validation accuracy: 11.7%
Mi

In [13]:
batch_size = 128
hidden_0 = 1024
hidden_1 = 2048

beta = 0.00001

decay_steps = 100
decay_rate = 0.90

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float64, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_0 = tf.Variable(
    tf.truncated_normal([feature_size, hidden_0], stddev=0.001))
  weights_1 = tf.Variable(
    tf.truncated_normal([hidden_0, hidden_1], stddev=0.001))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_1, num_labels], stddev=0.001))

  biases_0 = tf.Variable(tf.zeros([hidden_0]))
  biases_1 = tf.Variable(tf.zeros([hidden_1]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Training computation.
  learning_rate = tf.train.exponential_decay(0.3, global_step, decay_steps, decay_rate)
  
  logits_0 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_0) + biases_0)
  logits_1 = tf.nn.relu(tf.matmul(logits_0, weights_1) + biases_1)
  logits_2 = tf.matmul(logits_1, weights_2) + biases_2
  
  print(logits_2)
  print(tf_train_labels)
  loss = tf.reduce_mean(accuracy(logits_2,tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(weights_0) + beta * tf.nn.l2_loss(weights_1) + beta * tf.nn.l2_loss(weights_2)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      
  saver = tf.train.Saver()

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)

  valid_0 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_0) + biases_0)
  valid_1 = tf.nn.relu(tf.matmul(valid_0, weights_1) + biases_1)
  valid_2 = tf.matmul(valid_1, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(valid_2)

    
  #test_0 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_0) + biases_0)
  #test_1 = tf.nn.relu(tf.matmul(test_0, weights_1) + biases_1)
  #test_2 = tf.matmul(test_1, weights_2) + biases_2
  #test_prediction = tf.nn.softmax(test_2)

Tensor("add_2:0", shape=(128, 10), dtype=float32)
Tensor("Placeholder_1:0", shape=(128, 10), dtype=float64)


ValueError: axis(=1) out of bounds

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, w0, w1, w2 = session.run(
      [optimizer, loss, train_prediction, weights_0, weights_1, weights_2], feed_dict=feed_dict)
    if (step % 50 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

### Fresh attempt

In [5]:
batch_size = 64
hlSize0 = 2024
beta = 0.004
#decay_steps = 200
#decay_rate = 0.90
#learningStart=0.0007
decay_steps = 200
decay_rate = 0.98
learningStart=0.00007

stdv = 0.03
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,num_labels], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
    return layer2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  #loss = loss + beta * tf.nn.l2_loss(input_weights) + \
  #  beta * tf.nn.l2_loss(layer1_weights)
  #  beta * tf.nn.l2_loss(layer2_weights) + \
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()
  valid_prediction_val = valid_prediction.eval()

Initialized
Minibatch loss at step 0: 3.612277
Minibatch accuracy: 0.0%
Validation accuracy: 8.1%
Minibatch loss at step 100: 3.081617
Minibatch accuracy: 0.0%
Validation accuracy: 11.5%
Minibatch loss at step 200: 2.154027
Minibatch accuracy: 14.1%
Validation accuracy: 10.1%
Minibatch loss at step 300: 3.649023
Minibatch accuracy: 0.0%
Validation accuracy: 12.7%
Minibatch loss at step 400: 2.254417
Minibatch accuracy: 4.7%
Validation accuracy: 12.5%
Minibatch loss at step 500: 2.471145
Minibatch accuracy: 0.0%
Validation accuracy: 13.8%
Minibatch loss at step 600: 2.232859
Minibatch accuracy: 31.2%
Validation accuracy: 15.7%
Minibatch loss at step 700: 1.418697
Minibatch accuracy: 46.9%
Validation accuracy: 19.5%
Minibatch loss at step 800: 1.038151
Minibatch accuracy: 96.9%
Validation accuracy: 20.6%
Minibatch loss at step 900: 1.474093
Minibatch accuracy: 28.1%
Validation accuracy: 20.6%
Minibatch loss at step 1000: 2.948368
Minibatch accuracy: 7.8%
Validation accuracy: 22.5%
Miniba

In [7]:
validData = pd.DataFrame(valid_prediction_val)
Output.to_outputfile(validData,1,'NNfinalishValid')

In [11]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(firstHalfTest)

[[ 0.12555464  0.02739759  0.10776729 ...,  0.04831534  0.0630124
   0.2595064 ]
 [ 0.29324695  0.04345983  0.0160896  ...,  0.00503101  0.03180047
   0.14383648]
 [ 0.29037118  0.03378007  0.26029718 ...,  0.10769741  0.14575382
   0.02514734]
 ..., 
 [ 0.05630442  0.02067383  0.02101005 ...,  0.00987583  0.0020766
   0.01402664]
 [ 0.01897218  0.03959429  0.01928712 ...,  0.00068358  0.02324794
   0.08103255]
 [ 0.34150296  0.01651141  0.00100533 ...,  0.00097191  0.01560758
   0.32296211]]


In [22]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    thirdHalfTest = tf.nn.softmax(layer2).eval()
print(thirdHalfTest)

[[ 0.22806558  0.12997095  0.13495874 ...,  0.08459295  0.04555381
   0.0557115 ]
 [ 0.09180526  0.03840186  0.03776614 ...,  0.08799823  0.07087118
   0.47037387]
 [ 0.0542636   0.04385307  0.05653396 ...,  0.00983469  0.03361567
   0.24700759]
 ..., 
 [ 0.02237973  0.00690734  0.00548339 ...,  0.01811261  0.03638378
   0.41283646]
 [ 0.072421    0.04396648  0.02306206 ...,  0.29994926  0.19569038
   0.00929809]
 [ 0.12580341  0.01225203  0.00352421 ...,  0.01479877  0.5426181
   0.06715485]]


In [14]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    secondHalfTest = tf.nn.softmax(layer2).eval()
print(secondHalfTest)

[[  3.69661488e-02   3.39022502e-02   3.09263589e-03 ...,   2.77760089e-03
    1.14528432e-01   2.57804483e-01]
 [  1.83426782e-01   1.73101678e-01   3.62384878e-02 ...,   7.64091983e-02
    1.21946298e-01   1.09924920e-01]
 [  4.56958413e-02   2.12017186e-02   2.82088757e-01 ...,   3.06101225e-04
    1.67002767e-01   7.23533556e-02]
 ..., 
 [  3.07071675e-02   5.74692339e-02   6.35425821e-02 ...,   1.23854040e-03
    6.91337138e-02   1.00241728e-01]
 [  3.04357093e-02   1.16679333e-02   1.41384572e-01 ...,   3.32707077e-01
    2.70471632e-01   4.45370376e-02]
 [  6.08687289e-02   3.28430422e-02   3.75269470e-03 ...,   6.51790434e-03
    8.28824416e-02   1.94906190e-01]]


In [15]:
testClass = np.concatenate([firstHalfTest,secondHalfTest],0)
testClass = pd.DataFrame(testClass)
Output.to_outputfile(testClass,2,"NNfinalish",validation=False)

### Only classifying the first label

In [61]:
batch_size = 64
hlSize0 = 128
beta = 0.0001
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,1))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=0.01))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,num_labels], 
            stddev=0.01))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
  output_weights = tf.Variable(tf.truncated_normal(
      [num_labels,1],
            stddev=0.01))
  output_biases = 0
  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.nn.relu(tf.matmul(layer1, layer1_weights) + layer1_biases)
    layer3 = tf.matmul(layer2, output_weights) + output_biases
    return layer3
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits, tf_train_labels))
  #loss = loss + beta * tf.nn.l2_loss(input_weights) + \
  #  beta * tf.nn.l2_loss(layer1_weights) + \
  #  beta * tf.nn.l2_loss(layer2_weights) + \
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [65]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels0.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels0[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % singleAccuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % singleAccuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")

Initialized
Minibatch loss at step 0: 0.692604
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 100: 0.663300
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 200: 0.103247
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 300: 2.119647
Minibatch accuracy: 100.0%
Validation accuracy: 20.4%
Minibatch loss at step 400: 0.073297
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 500: 0.073943
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 600: 0.091196
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 700: 0.183572
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 800: 0.174383
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 900: 0.121723
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
Minibatch loss at step 1000: 0.057694
Minibatch accuracy: 0.0%
Validation accuracy: 20.4%
finished

### Classify

In [7]:
with tf.Session() as session:
    saver.restore(sess, "/tmp/model.ckpt")

NameError: name 'saver' is not defined

## Second attempt

In [104]:
batch_size = 64
hlSize0 = 2024
hlSize1 = 1024
beta = 0.05
decay_steps = 200
decay_rate = 0.90
learningStart=0.0007
stdv = 0.003
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,hlSize1], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[hlSize1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [hlSize1,num_labels], 
            stddev=stdv))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
    
  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.nn.relu(tf.matmul(layer1, layer1_weights) + layer1_biases)
    layer3 = tf.matmul(layer2, layer2_weights) + layer2_biases
    return layer3
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights) + \
    beta * tf.nn.l2_loss(layer2_weights)
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [105]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()

Initialized
Minibatch loss at step 0: 4.067645
Minibatch accuracy: 0.0%
Validation accuracy: 12.0%
Minibatch loss at step 100: 4.055054
Minibatch accuracy: 3.1%
Validation accuracy: 13.2%
Minibatch loss at step 200: 4.048203
Minibatch accuracy: 0.0%
Validation accuracy: 13.1%


KeyboardInterrupt: 

## 1x1 convolution?

In [6]:
batch_size = 64
hlSize0 = 2024
beta = 0.01
decay_steps = 200
decay_rate = 0.90
learningStart=0.0007
stdv = 0.03
patch_size = 1
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, num_labels], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [num_labels,num_labels], 
            stddev=stdv))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    print(tf.shape(layer1))
    #layer1 = tf.reshape(layer1,[hlSize0,1,1,1])
    conv = tf.nn.conv2d(layer1, layer1_weights, [1, 1], padding='SAME')
    layer2 = tf.nn.relu(conv + layer1_biases)
    layer3 = tf.matmul(layer2, layer2_weights) + layer2_biases
    return layer3
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights) + \
    beta * tf.nn.l2_loss(layer2_weights)
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

Tensor("Shape:0", shape=(2,), dtype=int32)


ValueError: Shape (64, 2024) must have rank 4

In [6]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()

Initialized
Minibatch loss at step 0: 36.045696
Minibatch accuracy: 0.0%
Validation accuracy: 11.7%
Minibatch loss at step 100: 36.571320
Minibatch accuracy: 3.1%
Validation accuracy: 16.9%
Minibatch loss at step 200: 31.054327
Minibatch accuracy: 40.6%
Validation accuracy: 25.9%
Minibatch loss at step 300: 28.318371
Minibatch accuracy: 100.0%
Validation accuracy: 25.8%
Minibatch loss at step 400: 29.402561
Minibatch accuracy: 73.4%
Validation accuracy: 22.4%
Minibatch loss at step 500: 29.327303
Minibatch accuracy: 32.8%
Validation accuracy: 29.0%
Minibatch loss at step 600: 30.051636
Minibatch accuracy: 28.1%
Validation accuracy: 33.8%
Minibatch loss at step 700: 28.164223
Minibatch accuracy: 100.0%
Validation accuracy: 30.9%
Minibatch loss at step 800: 29.349073
Minibatch accuracy: 35.9%
Validation accuracy: 37.3%
Minibatch loss at step 900: 28.201241
Minibatch accuracy: 96.9%
Validation accuracy: 42.9%
Minibatch loss at step 1000: 31.330561
Minibatch accuracy: 7.8%
Validation accur

## Added momentum

In [15]:
batch_size = 64
hlSize0 = 2024
hlSize1 = 1024
beta = 0.05
decay_steps = 100
decay_rate = 0.90
learningStart=0.0007
#learning_rate=0.001
stdv = 0.003
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,hlSize1], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[hlSize1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [hlSize1,num_labels], 
            stddev=stdv))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
    
  # Model.
  def model(data):
    layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.nn.relu(tf.matmul(layer1, layer1_weights) + layer1_biases)
    layer3 = tf.matmul(layer2, layer2_weights) + layer2_biases
    return layer3
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights) + \
    beta * tf.nn.l2_loss(layer2_weights)
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()

Initialized
Minibatch loss at step 0: 3.016086
Minibatch accuracy: 0.0%
Validation accuracy: 10.6%
Minibatch loss at step 100: 2.970623
Minibatch accuracy: 0.0%
Validation accuracy: 11.4%
Minibatch loss at step 200: 2.955587
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 300: 2.891662
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 400: 2.883225
Minibatch accuracy: 17.2%
Validation accuracy: 10.7%
Minibatch loss at step 500: 2.874598
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 600: 2.810019
Minibatch accuracy: 71.9%
Validation accuracy: 10.7%
Minibatch loss at step 700: 2.796701
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 800: 2.777732
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%

In [21]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    firstHalfTest = tf.nn.softmax(layer2).eval()
print(firstHalfTest)

[[ 0.00078307  0.00095813  0.00080282 ...,  0.0007355   0.00077737
   0.00073058]
 [ 0.00093672  0.00094288  0.00091188 ...,  0.00072089  0.00076084
   0.00071527]
 [ 0.00058513  0.00088732  0.00061786 ...,  0.0007136   0.00073604
   0.00071236]
 ..., 
 [ 0.00089966  0.00088711  0.0008768  ...,  0.00066718  0.00072292
   0.00066195]
 [ 0.00075745  0.00110351  0.00081773 ...,  0.00074436  0.00075917
   0.00074965]
 [ 0.00069846  0.00069995  0.00067106 ...,  0.00062636  0.00067104
   0.00062526]]


In [27]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    secondHalfTest = tf.nn.softmax(layer2).eval()
print(secondHalfTest)

[[ 0.00111596  0.00091727  0.00106503 ...,  0.0006479   0.00073936
   0.00065319]
 [ 0.00090069  0.00105173  0.00092015 ...,  0.00072383  0.00077124
   0.0007149 ]
 [ 0.00084777  0.00083204  0.0008227  ...,  0.00066847  0.00072262
   0.00066509]
 ..., 
 [ 0.00105296  0.00102638  0.00105567 ...,  0.00069359  0.00075461
   0.00069679]
 [ 0.0007897   0.00106825  0.00083355 ...,  0.00063956  0.00067962
   0.0006287 ]
 [ 0.0007754   0.00076975  0.00073611 ...,  0.00071936  0.00077459
   0.00071817]]


In [1]:
testClass = np.concatenate([firstHalfTest,secondHalfTest],0)
testClass = pd.DataFrame(testClass)
Output.to_outputfile(testClass,2,"NNtest3")

NameError: name 'np' is not defined

## with dropout

In [6]:
batch_size = 64
hlSize0 = 2000
hlSize1 = 1000
beta = 0.001
#decay_steps = 200
#decay_rate = 0.90
#learningStart=0.0007
decay_steps = 200
decay_rate = 0.90
learningStart=0.0007
keep_prob = 0.5

stdv = 0.03
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,hlSize1], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[hlSize1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [hlSize1,num_labels], 
            stddev=stdv))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
  
  # Model.
  def model(data,train):
    if (train == 1):
        layer1 = tf.nn.relu(tf.matmul(tf.nn.dropout(data, keep_prob), input_weights) + input_biases)
    else :
        layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
    layer3 = tf.matmul(layer2, layer2_weights) + layer2_biases
    return layer3
  
  # Training computation.
  logits = model(tf_train_dataset,1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights) + \
    beta * tf.nn.l2_loss(layer2_weights)
  #  beta * tf.nn.l2_loss(layer3_weights) + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  #learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  #optimizer = tf.train.AdadeltaOptimizer().minimize(loss)
  optimizer = tf.train.AdagradOptimizer(0.001).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset,0))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()
  layer2_biases_val = layer2_biases.eval()
  layer2_weights_val = layer2_weights.eval()
  valid_prediction_val = valid_prediction.eval()

Initialized
Minibatch loss at step 0: 7.311119
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 100: 8.242072
Minibatch accuracy: 3.1%
Validation accuracy: 14.9%
Minibatch loss at step 200: 5.363883
Minibatch accuracy: 25.0%
Validation accuracy: 28.2%
Minibatch loss at step 300: 4.963015
Minibatch accuracy: 25.0%
Validation accuracy: 17.8%
Minibatch loss at step 400: 4.990660
Minibatch accuracy: 25.0%
Validation accuracy: 30.7%
Minibatch loss at step 500: 4.696717
Minibatch accuracy: 29.7%
Validation accuracy: 32.4%
Minibatch loss at step 600: 5.082463
Minibatch accuracy: 20.3%
Validation accuracy: 34.8%
Minibatch loss at step 700: 4.118400
Minibatch accuracy: 50.0%
Validation accuracy: 29.1%
Minibatch loss at step 800: 5.142662
Minibatch accuracy: 25.0%
Validation accuracy: 32.3%
Minibatch loss at step 900: 3.764737
Minibatch accuracy: 82.8%
Validation accuracy: 35.4%
Minibatch loss at step 1000: 6.442114
Minibatch accuracy: 6.2%
Validation accuracy: 39.6%
Mi

In [8]:
#with tf.Session(graph=graph) as session:
#    layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases)
#    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
#    layer3 = tf.nn.softmax(tf.matmul(layer2, layer2_weights) + layer2_biases)
    #layer1 = tf.nn.relu(tf.matmul(valid_dataset, input_weights_val) + input_biases_val)
    #layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
#    validData = layer3.eval()

validData = pd.DataFrame(valid_prediction_val)
validData.to_csv('NN4validPredictions.csv')

In [11]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    layer3 = tf.matmul(layer2, layer2_weights_val) + layer2_biases_val
    firstHalfTest = tf.nn.softmax(layer3).eval()
print(firstHalfTest)

[[  4.84411931e-03   7.60621503e-02   2.29734927e-01 ...,   4.21042517e-02
    1.00717030e-01   1.78937957e-01]
 [  1.42821223e-01   1.02126941e-01   1.22120162e-03 ...,   2.20743148e-03
    4.38187905e-02   2.06218302e-01]
 [  4.55989063e-01   2.79622585e-01   4.11956757e-02 ...,   1.27872750e-02
    4.97623086e-02   1.54123932e-01]
 ..., 
 [  9.94103961e-03   7.82047585e-02   1.27162505e-02 ...,   3.07481340e-03
    1.91718638e-01   8.68645608e-02]
 [  8.97621829e-03   1.87869256e-04   6.29380907e-07 ...,   3.62184946e-04
    6.55896366e-02   8.98753777e-02]
 [  1.61460806e-02   5.18567741e-01   2.87237763e-03 ...,   1.07305276e-03
    2.03112084e-02   2.16739904e-02]]


In [14]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    layer3 = tf.matmul(layer2, layer2_weights_val) + layer2_biases_val
    firstHalfTest = np.concatenate([firstHalfTest,tf.nn.softmax(layer3).eval()],0)
#print(secondHalfTest)

In [17]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    layer3 = tf.matmul(layer2, layer2_weights_val) + layer2_biases_val
    firstHalfTest = np.concatenate([firstHalfTest,tf.nn.softmax(layer3).eval()],0)
#print(thirdHalfTest)

In [20]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    layer3 = tf.matmul(layer2, layer2_weights_val) + layer2_biases_val
    firstHalfTest = np.concatenate([firstHalfTest,tf.nn.softmax(layer3).eval()],0)
#print(fourthHalfTest)

In [21]:
#testClass = np.concatenate([firstHalfTest,secondHalfTest,thirdHalfTest,fourthHalfTest],0)
testClass = pd.DataFrame(firstHalfTest)
Output.to_outputfile(testClass,2,"NNtest4")

## moar layerzzz

In [5]:
batch_size = 64
hlSize0 = 2000
hlSize1 = 1000
hlSize2 = 500
beta = 0.001
#decay_steps = 200
#decay_rate = 0.90
#learningStart=0.0007
decay_steps = 200
decay_rate = 0.90
learningStart=0.0007
keep_prob = 0.5

stdv = 0.03
#patch_size = 5
#depth = 16
#num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  global_step = tf.Variable(0)  # count the number of steps taken.

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, feature_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(tf.truncated_normal(
      [feature_size,hlSize0],
            stddev=stdv))
  input_biases = tf.Variable(tf.zeros([hlSize0]))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [hlSize0,hlSize1], 
            stddev=stdv))
  layer1_biases = tf.Variable(tf.constant(0.0, shape=[hlSize1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [hlSize1,hlSize2], 
            stddev=stdv))
  layer2_biases = tf.Variable(tf.constant(0.0, shape=[hlSize2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [hlSize2,num_labels], 
            stddev=stdv))
  layer3_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
  
  
  # Model.
  def model(data,train):
    if (train == 1):
        layer1 = tf.nn.relu(tf.matmul(tf.nn.dropout(data, keep_prob), input_weights) + input_biases)
    else :
        layer1 = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
    layer3 = tf.matmul(layer2, layer2_weights) + layer2_biases
    layer4 = tf.matmul(layer3, layer3_weights) + layer3_biases
    return layer4
  
  # Training computation.
  logits = model(tf_train_dataset,1)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta * tf.nn.l2_loss(input_weights) + \
    beta * tf.nn.l2_loss(layer1_weights) + \
    beta * tf.nn.l2_loss(layer2_weights) + \
    beta * tf.nn.l2_loss(layer3_weights)# + \
  #  beta * tf.nn.l2_loss(output_weights)

  # Optimizer.
  #learning_rate = tf.train.exponential_decay(learningStart, global_step, decay_steps, decay_rate)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  #optimizer = tf.train.AdadeltaOptimizer().minimize(loss)
  optimizer = tf.train.AdagradOptimizer(0.001).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset,0))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  print("finished!")
  input_weights_val = input_weights.eval()
  input_biases_val = input_biases.eval()
  layer1_weights_val = layer1_weights.eval()
  layer1_biases_val = layer1_biases.eval()
  layer2_weights_val = layer2_weights.eval()
  layer2_biases_val = layer2_biases.eval()
  layer3_weights_val = layer3_weights.eval()
  layer3_biases_val = layer3_biases.eval()
  valid_prediction_val = valid_prediction.eval()

Initialized
Minibatch loss at step 0: 6.297700
Minibatch accuracy: 0.0%
Validation accuracy: 10.7%
Minibatch loss at step 100: 6.483749
Minibatch accuracy: 3.1%
Validation accuracy: 17.3%
Minibatch loss at step 200: 5.459052
Minibatch accuracy: 20.3%
Validation accuracy: 21.6%
Minibatch loss at step 300: 4.921887
Minibatch accuracy: 23.4%
Validation accuracy: 20.9%
Minibatch loss at step 400: 5.052115
Minibatch accuracy: 29.7%
Validation accuracy: 29.6%
Minibatch loss at step 500: 5.185280
Minibatch accuracy: 17.2%
Validation accuracy: 29.8%
Minibatch loss at step 600: 5.098585
Minibatch accuracy: 28.1%
Validation accuracy: 35.1%
Minibatch loss at step 700: 4.716326
Minibatch accuracy: 35.9%
Validation accuracy: 28.8%
Minibatch loss at step 800: 5.438993
Minibatch accuracy: 18.8%
Validation accuracy: 36.2%
Minibatch loss at step 900: 4.208798
Minibatch accuracy: 79.7%
Validation accuracy: 37.2%
Minibatch loss at step 1000: 6.145782
Minibatch accuracy: 6.2%
Validation accuracy: 41.2%
Mi

In [13]:
#with tf.Session(graph=graph) as session:
#    layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases)
#    layer2 = tf.matmul(layer1, layer1_weights) + layer1_biases
#    layer3 = tf.nn.softmax(tf.matmul(layer2, layer2_weights) + layer2_biases)
    #layer1 = tf.nn.relu(tf.matmul(valid_dataset, input_weights_val) + input_biases_val)
    #layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
#    validData = layer3.eval()

validData = pd.DataFrame(valid_prediction_val)
validData.to_csv('NN5validPredictions.csv')

In [15]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    layer3 = tf.matmul(layer2, layer2_weights_val) + layer2_biases_val
    layer4 = tf.matmul(layer3, layer3_weights_val) + layer3_biases_val
    testData = tf.nn.softmax(layer4).eval()
#print(firstHalfTest)

In [22]:
with tf.Session() as session:
    layer1 = tf.nn.relu(tf.matmul(test_data, input_weights_val) + input_biases_val)
    layer2 = tf.matmul(layer1, layer1_weights_val) + layer1_biases_val
    layer3 = tf.matmul(layer2, layer2_weights_val) + layer2_biases_val
    layer4 = tf.matmul(layer3, layer3_weights_val) + layer3_biases_val
    testData = np.concatenate([testData,tf.nn.softmax(layer4).eval()],0)
#print(firstHalfTest)

In [26]:
#testClass = np.concatenate([firstHalfTest,secondHalfTest,thirdHalfTest,fourthHalfTest],0)
testClass = pd.DataFrame(testData)
Output.to_outputfile(testClass,2,"NNtest5")